In [1]:
from flask import Flask, render_template, redirect, url_for, request, flash
import pandas as pd 
import numpy as np
import xlsxwriter
from werkzeug.utils import secure_filename
from dotenv import load_dotenv
import os

In [1]:
schedule_df = pd.read_csv('alarm_codes.csv', usecols= ['Error  Number', 'Error  Type'])
schedule_df = schedule_df.dropna()
schedule_df['Error  Number'] = schedule_df['Error  Number'].str.replace('Alm', '').astype('int64')

# Creating a flask web app instance
load_dotenv()
app = Flask(__name__)
app.secret_key = 'your_secret_key_here'
UPLOAD_FOLDER = ''
ALLOWED_EXTENSIONS = {'xlsx', 'csv'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
        return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_raw_data(filename):
        raw_df = pd.read_excel(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        schedule_df = pd.read_csv('data/alarm_codes.csv', usecols=['Error  Number', 'Error  Type'])
        schedule_df = schedule_df.dropna()
        schedule_df['Error  Number'] = schedule_df['Error  Number'].str.replace('Alm', '').astype('int64')

        # Create a dictionary mapping error codes to error types
        error_type_dict = dict(zip(schedule_df['Error  Number'], schedule_df['Error  Type']))
        raw_df['First Error Code'] = raw_df['Codes'].str.split(';').str[0]
        raw_df['First Error Code'] = raw_df['First Error Code'].fillna(0)

        # Assign error types to raw_df based on the dictionary
        raw_df['Error Type'] = raw_df['First Error Code'].astype('int64').map(error_type_dict).fillna('Unknown')
        excel_filename = 'Raw_Data_output.xlsx'
        writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')

        turb_dfs = {}

        for turbine_name, group_df in raw_df.groupby('WTG Number'):
                turb_dfs[turbine_name] = group_df

        for turbine, df in turb_dfs.items():
                penalizing_downtime = df[df['Error Type'] == '1']
                non_penalizing_downtime = df[df['Error Type'] == '0']
                warning = df[df['Error Type'] == 'W']
                unknown = df[df['Error Type'] == 'Unknown']

                penalizing_downtime.to_excel(writer, index=False, sheet_name=f'{turbine}_Penalising_Downtime', header=True)
                non_penalizing_downtime.to_excel(writer, index=False, sheet_name=f'{turbine}_Non_Penalising_Downtime', header=True)
                warning.to_excel(writer, index=False, sheet_name=f'{turbine}_Warning', header=True)
                unknown.to_excel(writer, index=False, sheet_name='Unknown Error Codes', header=True)

        writer.close()
        
        #excel_path = os.path.join(app.config['UPLOAD_FOLDER'], excel_filename)

        
        return excel_filename

def process_alarm_log(filename):
        alarm_log = pd.read_excel(os.path.join(app.config['UPLOAD_FOLDER'], filename), skiprows=2)
        alarm_log['From'] = pd.to_datetime(alarm_log['From'], format='mixed')
        alarm_log['To'] = pd.to_datetime(alarm_log['To'], format='mixed')
        
        alarm_log = alarm_log.dropna(subset=['To'])

        # Sort the data by 'From' timestamp
        alarm_log_sorted = alarm_log.sort_values(by='From')

        # Define a time window of 10 minutes
        time_window = pd.Timedelta(minutes=10)

        # Initialize variables to track overlapping errors
        start_time = None
        end_time = None
        selected_errors = []

        # Iterate over each row in the sorted data
        for _, row in alarm_log_sorted.iterrows():
                if start_time is None:
                        start_time = row['From']
                        end_time = row['To']
                        selected_error = row
                else:
                        if row['From'] - start_time <= time_window:
                        # If the error occurs within the time window, extend the end time
                                end_time = max(end_time, row['To'])
                        # Update the selected error with the latest end time
                                selected_error = row
                        else:
                                # If the error is outside the time window, select the previous error
                                selected_errors.append(selected_error)
                                # Start a new time window with the current error
                                start_time = row['From']
                                end_time = row['To']
                                selected_error = row

        # Select the last error
        if selected_error is not None:
                selected_errors.append(selected_error)
        
        selected_errors_df = pd.DataFrame(selected_errors)
        
        excel_filename = 'MarchAlarmOverlapFlask.xlsx'
        excel_path = os.path.join(app.config['UPLOAD_FOLDER'], excel_filename)
        selected_errors_df.to_excel(excel_path, index=False, header=True)

        return excel_filename

@app.route('/', methods=['GET', 'POST'])
def upload_file():
        if request.method == 'POST':
                if 'file' not in request.files:
                        flash('No file part')
                        return redirect(request.url)
                file = request.files['file']
                if file.filename == '':
                        flash('No selected file')
                        return redirect(request.url)
                if file and allowed_file(file.filename):
                        filename = secure_filename(file.filename)
                        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                        if 'raw_data' in filename:
                                result = process_raw_data(filename)
                        elif 'alarm_log' in filename:
                                result = process_alarm_log(filename)
                        else:
                                result = 'File uploaded successfully'
                        return render_template('success.html', message=result)
                else:
                        return render_template('error.html', message='Invalid file format')
        return render_template('index.html')

@app.route('/upload_raw_data', methods=['POST'])
def upload_raw_data():
        if 'file' not in request.files:
                flash('No file part')
                return redirect(request.url)
        
        file = request.files['file']
        if file.filename == '':
                flash('No selected file')
                return redirect(request.url)
        
        if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                
                # Call the processing function after the file is saved
                result = process_raw_data(filename)
                
                # Return a response indicating success
                return render_template('success.html', message=result)
        else:
                return render_template('error.html', message='Invalid file format')

@app.route('/upload_alarm_log', methods=['POST'])
def upload_alarm_log():
        if 'file' not in request.files:
                flash('No file part')
                return redirect(request.url)
        
        file = request.files['file']
        if file.filename == '':
                flash('No selected file')
                return redirect(request.url)
        
        if file and allowed_file(file.filename):
                filename = secure_filename(file.filename)
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                
                # Call the processing function after the file is saved
                result = process_alarm_log(filename)
                
                # Return a response indicating success
                return render_template('success.html', message=result)
        else:
                return render_template('error.html', message='Invalid file format')


if __name__ == '__main__':
        app.run()

NameError: name 'pd' is not defined

: 

In [12]:
schedule_df = pd.read_csv("alarm_codes.csv", usecols= ['Error  Number', 'Error  Type'])
schedule_df = schedule_df.dropna()
schedule_df['Error  Number'] = schedule_df['Error  Number'].str.replace('Alm', '').astype('int64')

# alog = alarm_log.sort_values(by='Duration(Seconds)', ascending=False)
# alog


In [15]:
alarm_log = pd.read_excel('March Data/NEW UK Alarm Log_28_02_2024_to_02_04_2024.xlsx', skiprows=2)

# Convert 'From' and 'To' columns to datetime objects
alarm_log['From'] = pd.to_datetime(alarm_log['From'], format='mixed')
alarm_log['To'] = pd.to_datetime(alarm_log['To'], format='mixed')


# Sort the data by 'From' timestamp
alarm_log_sorted = alarm_log.sort_values(by='From')

# Define a time window of 10 minutes
time_window = pd.Timedelta(minutes=10)

# Initialize variables to track overlapping errors
start_time = None
end_time = None
selected_errors = []

# Iterate over each row in the sorted data
for _, row in alarm_log_sorted.iterrows():
    if start_time is None:
        start_time = row['From']
        end_time = row['To']
        selected_error = row
    else:
        if row['From'] - start_time <= time_window:
            # If the error occurs within the time window, extend the end time
            end_time = max(end_time, row['To'])
            # Update the selected error with the latest end time
            selected_error = row
        else:
            # If the error is outside the time window, select the previous error
            selected_errors.append(selected_error)
            # Start a new time window with the current error
            start_time = row['From']
            end_time = row['To']
            selected_error = row

# Select the last error
if selected_error is not None:
    selected_errors.append(selected_error)

# Create a DataFrame containing selected errors
selected_errors_df = pd.DataFrame(selected_errors)

# Print the resulting DataFrame
selected_errors_df

writer = pd.ExcelWriter('March Data/MarchAlarmOverlap.xlsx', engine='xlsxwriter')

selected_errors_df.to_excel(writer, index=False, header=True)
    
writer.close() 

FileNotFoundError: [Errno 2] No such file or directory: 'March Data/NEW UK Alarm Log_28_02_2024_to_02_04_2024.xlsx'

In [13]:
# # Check if the errors with durations of 13 hours and 151 hours are included in selected_errors_df
# error_13_hours_included = (selected_errors_df['Duratoion Edit '] == '13:33:32').any()
# error_151_hours_included = (selected_errors_df['Duratoion Edit '] == '151:49:10').any()

# # Print the result
# print("Error with duration 13 hours included:", error_13_hours_included)
# print("Error with duration 151 hours included:", error_151_hours_included)

selected_errors_df_sorted = selected_errors_df.sort_values(by='Duration(Seconds)', ascending=False)
selected_errors_df_sorted


,From,To,Duration(Seconds),Group,Station,Communication Id,Alarm Code,LogType,Description,Additional Text,User,Comment
28903,2024-03-02 13:23:12.880,2024-03-21 23:18:48.537,1677336,Turbine,WTG08,6000622,122000,0,Intruder warning,NaN,NaN,NaN
27339,2024-03-07 12:28:18.460,2024-03-25 11:43:56.787,1552538,Turbine,WTG01,6000615,122000,0,Intruder warning,NaN,NaN,NaN
20542,2024-03-12 11:11:00.917,2024-03-25 13:24:11.617,1131191,Turbine,WTG03,6000617,122000,0,Intruder warning,NaN,NaN,NaN
13213,2024-03-19 11:33:48.497,2024-03-30 08:17:08.287,938600,Turbine,WTG05,6000619,104080,0,Gearbox oil temperature is low,NaN,NaN,NaN
27398,2024-03-06 11:28:56.250,2024-03-14 11:23:58.193,690902,Turbine,WTG06,6000620,122000,0,Intruder warning,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
20509,2024-03-12 12:45:54.983,NaT,0,Turbine,WTG04,6000618,180481,1,"Alarm Reactivation, Alarm:",114043,NaN,NaN
20516,2024-03-12 12:20:37.717,NaT,0,Turbine,WTG04,6000618,180481,1,"Alarm Reactivation, Alarm:",160103,NaN,NaN
20530,2024-03-12 11:59:21.083,NaT,0,Turbine,WTG02,6000616,180481,1,"Alarm Reactivation, Alarm:",122000,NaN,NaN
20534,2024-03-12 11:51:48.427,NaT,0,Turbine,WTG02,6000616,180481,1,"Alarm Reactivation, Alarm:",122000,NaN,NaN


In [14]:
writer = pd.ExcelWriter('test/test.xlsx', engine='xlsxwriter')

selected_errors_df.to_excel(writer, index=False, header=True)
    
writer.close() 

In [16]:
merged_df

,Project,WTG Number,WTG Name,time utc,Codes,errror time [minutes],windspeed,Energy [kWh],Production 1 [kWh],Consumption [kWh],...,Voltage 3 [V],Temperature Outside [°C],angle of attack pitch 1,angle of attack pitch 2,angle of attack pitch 3,Generator frequency [Hz],Gen reactive power[kVAr],Error Number,Description,Error Type
0,Longhill,WEA 1,6000615.0,2024-04-04 01:10:00,102092,1.0,8.3981,523.0,9366922.0,19448.0,...,400.5873,1.7000,-0.7016,-0.7055,-0.7095,49.9546,-939.6949,NaN,NaN,NaN
1,Longhill,WEA 1,6000615.0,2024-04-06 01:10:00,102092,1.0,4.4556,82.0,9487921.0,19451.0,...,401.0949,8.3000,0.3659,0.3613,0.3586,49.9978,-705.4774,NaN,NaN,NaN
2,Longhill,WEA 1,6000615.0,2024-04-13 01:10:00,102092,1.0,13.3992,1066.0,10120879.0,19738.0,...,403.7208,7.9000,9.3800,9.3796,9.3719,49.9971,-102.9535,NaN,NaN,NaN
3,Longhill,WEA 1,6000615.0,2024-04-14 21:50:00,102092,1.0,14.3777,1061.0,10375355.0,19738.0,...,402.5007,4.6533,11.0401,11.0116,11.0163,49.9373,-165.0674,NaN,NaN,NaN
4,Longhill,WEA 1,6000615.0,2024-04-22 14:40:00,102092,10.0,4.8299,83.0,10878133.0,20819.0,...,398.4718,10.9000,0.3227,0.3192,0.3168,49.9831,-947.1682,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987911,Longhill,WEA 8,6000622.0,2024-04-30 21:50:00,NaN,0.0,8.2621,492.0,5394941.0,13832.0,...,400.1177,11.2000,-0.9082,-0.8928,-0.8994,50.0074,-728.9904,NaN,NaN,NaN
5987912,Longhill,WEA 8,6000622.0,2024-04-30 21:50:00,NaN,0.0,8.2621,492.0,5394941.0,13832.0,...,400.1177,11.2000,-0.9082,-0.8928,-0.8994,50.0074,-728.9904,NaN,NaN,NaN
5987913,Longhill,WEA 8,6000622.0,2024-04-30 21:50:00,NaN,0.0,8.2621,492.0,5394941.0,13832.0,...,400.1177,11.2000,-0.9082,-0.8928,-0.8994,50.0074,-728.9904,NaN,NaN,NaN
5987914,Longhill,WEA 8,6000622.0,2024-04-30 21:50:00,NaN,0.0,8.2621,492.0,5394941.0,13832.0,...,400.1177,11.2000,-0.9082,-0.8928,-0.8994,50.0074,-728.9904,NaN,NaN,NaN


In [2]:

# raw_df = raw_df.drop(columns=raw_df.columns[0])
# raw_df = raw_df.dropna(axis=0, how='all', subset=raw_df.columns[1:])
# raw_df = raw_df.reset_index(drop=True)
raw_df = 
# # # Merge merged_df with schedule_df based on 'Error Code' and 'Error Number' columns
# merged_df = pd.merge(raw_df, schedule_df, left_on='Error Code', right_on='Error  Number', how='left')

# # Drop the redundant 'Error Number' column after merging
# merged_df.drop(columns='Error  Number', inplace=True)

# merged_df


NameError: name 'raw_df' is not defined

In [11]:
error_nan_df = merged_df[merged_df['Error  Type'].isna()]
grouped_error_codes = error_nan_df.groupby('Error Code')['Error Code'].unique()
grouped_error_codes

Error Code
102321    [102321]
102323    [102323]
103131    [103131]
103136    [103136]
104058    [104058]
104080    [104080]
105065    [105065]
106240    [106240]
106242    [106242]
106244    [106244]
106294    [106294]
107053    [107053]
107054    [107054]
110203    [110203]
110204    [110204]
110205    [110205]
110230    [110230]
110233    [110233]
110234    [110234]
113123    [113123]
113124    [113124]
113129    [113129]
160000    [160000]
160011    [160011]
160015    [160015]
160016    [160016]
160017    [160017]
160102    [160102]
160103    [160103]
160104    [160104]
160106    [160106]
Name: Error Code, dtype: object

In [29]:
merged_df_nona = merged_df.dropna(subset=['Alarm Code'])
turb_dfs_nona = {}

writer = pd.ExcelWriter('March Data/March_output_nona.xlsx', engine='xlsxwriter')

for turbine_name, group_df in merged_df_nona.groupby('Turbine Name'):
    turb_dfs_nona[turbine_name] = group_df
    turb_dfs_nona[turbine_name].to_excel(writer, sheet_name=turbine_name, index=False, header=True)
    
writer.close()

# if __name__ = '__main__':
#     app.run()  



In [7]:
# schedule_df['Error  Number'] = schedule_df['Error  Number'].astype('int32')

turb_dfs = {}

# writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

for turbine_name, group_df in raw_df.groupby('WTG Number'):
    turb_dfs[turbine_name] = group_df
    # turb_dfs[turbine_name].to_excel(writer, sheet_name=turbine_name, index=False, header=True)
    
    
# writer.close()




In [9]:
turb_dfs['WEA 1'].columns

Index(['Project', 'WTG Number', 'WTG Name', 'time utc', 'Codes',
       'errror time [minutes]', 'windspeed', 'Energy [kWh]',
       'Production 1  [kWh]', 'Consumption [kWh]', 'Power [kW]',
       'Power (min) [kW]', 'Power (max) [kW]', 'Nacelle direction [degree]',
       'Current 1 [A]', 'Current 2 [A]', 'Current 3 [A]', 'Voltage 1 [V]',
       'Voltage 2 [V]', 'Voltage 3 [V]', 'Temperature Outside [°C]',
       'angle of attack pitch 1', 'angle of attack pitch 2 ',
       'angle of attack pitch 3', 'Generator frequency [Hz]',
       'Gen reactive power[kVAr]'],
      dtype='object')

In [ ]:
# # Check unique values in 'Error Code' column of the turbine DataFrame
# unique_error_codes_turb = turb_dfs.get('WTG01')['Error Code'].unique()

# # Check unique values in 'Error Number' column of the schedule DataFrame
# unique_error_numbers_schedule = schedule_df['Error  Number'].unique()

# # Print unique values
# print("Unique Error Codes in Turbine DataFrame:", unique_error_codes_turb)
# print("Unique Error Numbers in Schedule DataFrame:", unique_error_numbers_schedule)


Unique Error Codes in Turbine DataFrame: [122000 108027 161423 109007 125005 106242 160104 160102 102323 114301
 110205 103088 114043 122014 123005 104080 117038 106244 107054 110019
 110203 160106 108054 103000 160016 106240 103015 102092 114062 106241
 114212 113123 122001 126028 106294 152311 160017 110063 107052 105043
 115002 160103 107032 120002 101008 122031 110204 126024 104083 125028
 123001 161638 161601 160015 110001]
Unique Error Numbers in Schedule DataFrame: [100001 100002 100003 ... 199125 199126 199999]


In [ ]:
# # Extract unique error codes from raw_df
# unique_error_codes_raw = raw_df['Error Code'].unique()

# # Extract unique error numbers from schedule_df
# unique_error_numbers_schedule = schedule_df['Error  Number'].unique()

# # Find error codes in raw_df that don't have a matching error number in schedule_df
# missing_error_codes = unique_error_codes_raw[~np.isin(unique_error_codes_raw, unique_error_numbers_schedule)]

# missing_error_codes


Here I will start writing out the calculations that will be performed on each turbine to calculate the Monthly Average Availability

Availability Guarantee  = 1 - (Penalised Hours / All Hours)

In [ ]:
# turb_dfs['WTG01'][['Turbine Name','Error Code', 'Error  Number', 'Error  Type']]

,Turbine Name,Error Code,Error Number,Error Type
0,WTG01,122000,122000.0,W
8,WTG01,122000,122000.0,W
16,WTG01,122000,122000.0,W
24,WTG01,122000,122000.0,W
32,WTG01,122000,122000.0,W
...,...,...,...,...
69962,WTG01,106294,NaN,NaN
69970,WTG01,106294,NaN,NaN
69978,WTG01,106294,NaN,NaN
69986,WTG01,160104,NaN,NaN


Showcase to tony and alex tomorrow. Filters by error type and outputs the excel sheets.

In [31]:
for turbine, df in turb_dfs_nona.items():
    print(f"Number of rows in {turbine} DataFrame before filtering: {len(df)}")
    penalizing_downtime = turb_nona[turb_nona['Error Type'] == '1']
    print(f"Number of rows with Error Type '1' in {turbine}: {len(penalizing_downtime)}")
    non_penalizing_downtime = turb_nona[turb_nona['Error Type'] == '0']
    print(f"Number of rows with Error Type '0' in {turbine}: {len(non_penalizing_downtime)}")
    warning = turb_nona[turb_nona['Error Type'] == 'w']
    print(f"Number of rows with Error Type 'w' in {turbine}: {len(warning)}")



Number of rows in WTG01 DataFrame before filtering: 11010


KeyError: ['Error Type']

In [ ]:
# pivot_df = pd.pivot_table(available_df, values='Fault_Count', index='Fault_type', columns='Turbine_Name', aggfunc='sum', fill_value=0)

# pivot_df.to_csv('output1.csv')
# type(turb_dfs['WTG01']['Date Time'][0])

This is the code attempting to aggregate the time and produce the start and end time using a pivot table 

In [ ]:
# writer = pd.ExcelWriter('output2.xlsx', engine='xlsxwriter')

# for turbine, df in turb_dfs.items():
#     turb_nona = df.dropna(subset=['Error  Type'])
#     df['Date Time'] = pd.to_datetime(df['Date Time'])
#     df = df.sort_values(by=['Date Time'])
    
#     df['New_Block'] = (df['Error Code'] != df['Error Code'].shift())
#     # Create a grouping variable to identify consecutive blocks of the same error code
#     df['Group'] = (df['New_Block'] == 1).cumsum()

#     agg_df = df.groupby(['Turbine Name', 'Error Code', 'Error  Type', 'Group']).agg(
#         Start_Time=('Date Time', 'min'),
#         End_Time=('Date Time', 'max')
#     ).reset_index()

#     agg_df.reset_index(inplace=True)

#     pivot_df = agg_df.pivot_table(index=['Turbine Name', 'Error Code', 'Error  Type'], columns=None, values=['Start_Time', 'End_Time'], aggfunc='first')
#     pivot_df.reset_index(inplace=True)

#     for index, row in pivot_df.iterrows():
#         turbine_name = row['Turbine Name']
#         error_type = row['Error  Type']

#         filtered_data = agg_df[(agg_df['Turbine Name'] == turbine_name) & (agg_df['Error  Type'] == error_type)]

#         filtered_data.to_excel(writer, index=False, sheet_name=f'{turbine_name}_{error_type}', header=True)

# writer.close()

In [57]:
# writer = pd.ExcelWriter('aggoutput1.xlsx', engine='xlsxwriter')
# for turbine, df in turb_dfs.items():
#     df = df.sort_values(by=['Date Time'])
#     turb_nona = df.dropna()
#     start_time = None
#     end_time = None
#     processed_data = []
#     for i in range(len(turb_nona) - 1):
#         current_row = turb_nona.iloc[i]
#         next_row = turb_nona.iloc[i + 1]
#         # Take the starting position i['Date Time'] as start time
#         if start_time is None:
#             start_time = current_row['Date Time']
#         # Check if the next row has a different error code
#         if current_row['Error Code'] != next_row['Error Code']:
#             end_time = current_row['Date Time']
#             # Filter rows except start and end times
#             filtered_rows = turb_nona[(turb_nona['Date Time'] == start_time) | (turb_nona['Date Time'] == end_time)]
#             processed_data.append(filtered_rows)
#             # Reset start_time for the new error code
#             start_time = None
#     # Process the last error code
#     if start_time is not None:
#         end_time = turb_nona.iloc[-1]['Date Time']
#         filtered_rows = turb_nona[(turb_nona['Date Time'] == start_time) | (turb_nona['Date Time'] == end_time)]
#         processed_data.append(filtered_rows)
        
#     result_df = pd.concat(processed_data)
#     result_df.to_excel(writer, index=False, sheet_name=f'{turbine_name}_{error_type}', header=True)
# writer.close()